# [Day 9: Rope Bridge](https://adventofcode.com/2022/day/9)

In [1]:
example = """R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2"""

Parse motions from input.

In [2]:
def parse(input):
    """Returns series of motions parsed from input."""
    return [
        (direction, int(steps)) 
        for direction, steps in [
            line.split() for line in input.splitlines()
        ]
    ]

parse(example)

[('R', 4),
 ('U', 4),
 ('L', 3),
 ('D', 1),
 ('R', 4),
 ('D', 1),
 ('L', 5),
 ('R', 2)]

Process a single step.

In [3]:
def step(head, tail, direction):
    """Returns position of head and tail after step in direction."""
    head_x, head_y = head
    tail_x, tail_y = tail
    match direction:
        case 'U':
            head_y += 1
        case 'D':
            head_y -= 1
        case 'L':
            head_x -= 1
        case 'R':
            head_x += 1
            
    if abs(head_x - tail_x) > 1:
        tail_x += 1 if head_x > tail_x else -1
        tail_y = head_y
    if abs(head_y - tail_y) > 1:
        tail_y += 1 if head_y > tail_y else -1
        tail_x = head_x

    return (head_x, head_y), (tail_x, tail_y)


# Tail should follow if head is two steps up, down, left or right.
assert step((1, 0), (0, 0), 'R') == ((2, 0), (1, 0))
assert step((-1, 0), (0, 0), 'L') == ((-2, 0), (-1, 0))
assert step((0, 1), (0, 0), 'U') == ((0, 2), (0, 1))
assert step((0, -1), (0, 0), 'D') == ((0, -2), (0, -1))

# Tail should follow diagonally.
assert step((1, 1), (0, 0), 'U') == ((1, 2), (1, 1))

# Tail shouldn't move if head moves on top of tail.
assert step((1, 0), (0, 0), 'L') == ((0, 0), (0, 0))

Track tail positions during simulation.

In [4]:
def simulate(state, motions):
    """Yields tail positions while simulating series of motions."""
    head, tail = state
    for direction, steps in motions:
        while steps:
            yield tail
            head, tail = step(head, tail, direction)
            steps -= 1
    yield tail

list(simulate(((0, 0), (0, 0)), parse(example)))

[(0, 0),
 (0, 0),
 (1, 0),
 (2, 0),
 (3, 0),
 (3, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 3),
 (3, 4),
 (2, 4),
 (2, 4),
 (2, 4),
 (2, 4),
 (3, 3),
 (4, 3),
 (4, 3),
 (4, 3),
 (4, 3),
 (3, 2),
 (2, 2),
 (1, 2),
 (1, 2),
 (1, 2)]

Count unique tail positions.

In [5]:
len(set(simulate(((0, 0), (0, 0)), parse(example))))

13

# Part 1

Count unique tail positions in input.

In [6]:
len(set(simulate(((0, 0), (0, 0)), parse(open('day-9-input.txt').read()))))

6181

# Part 2

Adapt `step()` to handle any number of knots.

In [7]:
def step(state, direction):
    """Returns state after step in direction."""
    head_x, head_y = state[0]
    next_state = [None] * len(state)
    
    # Move head.
    match direction:
        case 'U':
            head_y += 1
        case 'D':
            head_y -= 1
        case 'L':
            head_x -= 1
        case 'R':
            head_x += 1
    next_state[0] = (head_x, head_y)
    
    # Move remaining knots in body.
    for knot in range(len(state) - 1):
        head_x, head_y = next_state[knot]
        tail_x, tail_y = state[knot + 1]
        # Tail follows diagonally when head moves diagonally.
        if abs(head_x - tail_x) > 1 and abs(head_y - tail_y) > 1:
            tail_x += 1
            tail_y += 1
        elif abs(head_x - tail_x) > 1:
            tail_x += 1 if head_x > tail_x else -1
            tail_y = head_y
        elif abs(head_y - tail_y) > 1:
            tail_y += 1 if head_y > tail_y else -1
            tail_x = head_x
        next_state[knot + 1] = (tail_x, tail_y)

    return next_state

# Tail should follow if head is two steps up, down, left or right.
assert step([(1, 0), (0, 0)], 'R') == [(2, 0), (1, 0)]
assert step([(-1, 0), (0, 0)], 'L') == [(-2, 0), (-1, 0)]
assert step([(0, 1), (0, 0)], 'U') == [(0, 2), (0, 1)]
assert step([(0, -1), (0, 0)], 'D') == [(0, -2), (0, -1)]

# Tail should follow diagonally.
assert step([(1, 1), (0, 0)], 'U') == [(1, 2), (1, 1)]

# Tail shouldn't move if head moves on top of tail.
assert step([(1, 0), (0, 0)], 'L') == [(0, 0), (0, 0)]

# Knots should move diagonally if body knots move diagonally.
assert step([(2, 2), (1, 1), (0, 0)], 'U') == [(2, 3), (2, 2), (1, 1)]

Extend simulate to handle rope with any number of knots.

In [25]:
def simulate(state, motions):
    """Yields state while simulating series of motions."""
    for direction, steps in motions:
        while steps:
            state = step(state, direction)
            steps -= 1
        yield state
    yield state

Simulate rope with 10 knots on example input. Count unique tail positions.

In [26]:
def visualise(states):
    min_x = min(knot[0] for state in states for knot in state)
    max_x = max(knot[0] for state in states for knot in state)
    min_y = min(knot[1] for state in states for knot in state)
    max_y = max(knot[1] for state in states for knot in state)

    for state in states:
        canvas = [['.'] * (max_x - min_x + 1) for _ in range(max_y - min_y + 1)]
        canvas[len(canvas) - 1 - (0 - min_y)][0 - min_x] = 's'
        for index, (x, y) in enumerate(reversed(state)):
            canvas[len(canvas) - 1 - (y - min_y)][x - min_x] = str(len(state) - index - 1)
#             canvas[len(canvas) - 1 - (y - min_y)][x - min_x] = '#'
        print('\n' + '\n'.join(''.join(row) for row in canvas))

visualise(list(simulate([(0, 0)] * 10, parse(example))))


......
......
......
......
43210.

....0.
....1.
..432.
.5....
6.....

.01...
...2..
..43..
.5....
6.....

..1...
.0.2..
..43..
.5....
6.....

......
...210
..43..
.5....
6.....

......
...21.
..43.0
.5....
6.....

......
......
0123..
.5....
6.....

......
......
.103..
.5....
6.....

......
......
.103..
.5....
6.....


Count unique tail positions.

In [10]:
len(set(state[-1] for state in simulate([(0, 0)] * 10, parse(example))))

1

In [11]:
larger_example = """R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20"""

Simulate rope with 10 knots on larger example. Count unique tail positions.

In [12]:
len(set(state[-1] for state in simulate([(0, 0)] * 10, parse(larger_example))))

36

Simulate rope with 10 knots on input. Count unique tail positions.

In [28]:
len(set(state[-1] for state in simulate([(0, 0)] * 10, parse(open('day-9-input.txt').read()))))

1957

In [27]:
visualise(list(simulate([(0, 0)] * 10, parse(larger_example))))


..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........543210.........
..........................
..........................
..........................
..........................
..........................

..........................
..........................
..........................
..........................
..........................
..........................
..........................
................0.........
................1.........
................2.........
................3.........
...............54.........
..............6...........
.............7............
............8.............
...........9..............

In [13]:
visualise([[state[-1] for state in simulate([(0, 0)] * 10, parse(larger_example))]])


#.....................
#.............###.....
#............#...#....
.#..........#.....#...
..#..........#.....#..
...#........#.......#.
....#......#.........#
.....#..............#.
......#............#..
.......#..........#...
........#........#....
.........########.....
